In [ ]:
!pip install nltk==3.4
!pip install yellowbrick -U
!pip install gensim==3.6.0

In [8]:
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Plotly imports
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
py.init_notebook_mode(connected=True)

# Other imports
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import json

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans



import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200) # to show more rows.

%matplotlib inline
%load_ext autoreload
%autoreload 2

References :
1. https://www.kaggle.com/arthurtok/spooky-nlp-and-topic-modelling-tutorial
2. https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
3. https://github.com/ElizaLo/NLP-Natural-Language-Processing

In [3]:
df = pd.read_json('../input/careerdb/database.json')

In [5]:
df.rename(columns={'pregrado':'undergraduate',
                   'Universidad':'university',
                   'detalles':'details',
                   'Descripcion':'description',
                  'Registro calificado':'Qualified record',
                  'Nivel de formación':'Level of Education',
                  'Tipo de formación':'Type of training',
                  'Título otorgado':'Title awarded',
                  'Modalidad':'Modality',
                  'Duración':'Duration',
                  'Créditos':'Credits',
                  'Ciudad':'Town'},inplace=True)
df.head()

In [6]:
def nan_check(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data

nan_check(df)

In [ ]:
df.info()

In [12]:
df.undergraduate.value_counts()

In [14]:
# Delete .....
df1 = df[(df['undergraduate'].str.find('especializacion')==-1) &
    (df['undergraduate'].str.find('maestria')==-1) & 
    (df['undergraduate'].str.find('doctorado')==-1) & 
    (df['undergraduate'].str.find('tecnologia')==-1)]

df1.info()

In [ ]:
df1.undergraduate.value_counts()

In [15]:
def replace_str(x):
    if (str(x).find('administracion') != -1) or (str(x).find('contaduria') != -1) \
    or (str(x).find('economia') != -1) or (str(x).find('negocios') != -1) \
    or (str(x).find('mercadeo') != -1) or (str(x).find('finanzas') != -1) \
    or (str(x).find('publicidad') != -1) or (str(x).find('comercio') != -1):
        return 'BUSINESS'
    elif str(x).find('ingenieria') != -1:
        return 'ENGINEERING'
    elif (str(x).find('derecho') != -1)   or (str(x).find('psico') != -1) \
    or (str(x).find('comunicacion') != -1) or (str(x).find('social') != -1) \
    or (str(x).find('pedagogia') != -1) or (str(x).find('filosofia') != -1) \
    or (str(x).find('educacion infantil') != -1) or (str(x).find('teologia') != -1) \
    or (str(x).find('antropologia') != -1) or (str(x).find('ciencia politica') != -1) \
    or (str(x).find('historia') != -1) or (str(x).find('sociologia') != -1) \
    or (str(x).find('literatura') != -1) or (str(x).find('ciencias politicas') != -1) \
    or (str(x).find('relaciones internacionales') != -1):
        return 'HUMANITIES AND SOCIAL SCIENCE'
    elif (str(x).find('medicina') != -1) or (str(x).find('enfermeria') != -1) \
    or (str(x).find('odontologia') != -1) or (str(x).find('salud en el trabajo') != -1) \
    or (str(x).find('quirurgica') != -1) :
        return 'HEALTH & MEDICINE'
    elif (str(x).find('educacion fisica') != -1) or (str(x).find('fisioterapia') != -1):
        return 'SPORTS AND PHYSICAL TRAIN'
    elif (str(x).find('arquitectura') != -1) or (str(x).find('music') != -1) \
    or (str(x).find('diseño') != -1) or (str(x).find('artes') != -1) \
    or (str(x).find('fotografia') != -1):
        return 'ARTS AND DESIGN'
    elif (str(x).find('matematicas') != -1) or (str(x).find('fisica') != -1) \
    or (str(x).find('estadistica') != -1) or (str(x).find('biologia') != -1) \
    or (str(x).find('ciencias naturales') != -1) or (str(x).find('quimica') != -1) :
        return 'MATH AND PHYSICAL SCIENCES'
    else:
        return 'OTHER'

In [16]:
df1['final_cat'] = df1['undergraduate'].apply(replace_str)
df1['final_cat'].value_counts(normalize = True)

In [18]:
# Droping OTHER Undergraduate
df1 = df1[df1['final_cat'] != 'OTHER']
# Dropping void Description
df1= df1[df1['description']!='']
# replacing big space
df1['description'] = df1['description'].str.strip('').replace('  ', ' ')

In [19]:
df1.head(2)

In [20]:
data = [go.Bar(
            x = df1.final_cat.unique(),
            y = df1.final_cat.value_counts().values,
            marker= dict(colorscale='Jet',
                         color = df1.final_cat.value_counts().values
                        ),
            text='Text entries attributed to Final Category'
    )]

layout = go.Layout(
    title='Target variable distribution'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [21]:
data = [go.Bar(
            x = df1.undergraduate.unique(),
            y = df1.undergraduate.value_counts().values,
            marker= dict(colorscale='Jet',
                         color = df1.undergraduate.value_counts().values
                        ),
            text='Text entries attributed to undergraduate Category'
    )]

layout = go.Layout(
    title='Undergraduate variable distribution'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [22]:
all_words = df1['undergraduate'].str.split(expand=True).unstack().value_counts()
data = [go.Bar(
            x = all_words.index.values[2:50],
            y = all_words.values[2:50],
            marker= dict(colorscale='Jet',
                         color = all_words.values[2:100]
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 (Uncleaned) Word frequencies in the Undergraduate'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [23]:
all_words = df1['description'].str.split(expand=True).unstack().value_counts()
data = [go.Bar(
            x = all_words.index.values[2:50],
            y = all_words.values[2:50],
            marker= dict(colorscale='Jet',
                         color = all_words.values[2:100]
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 (Uncleaned) Word frequencies in the Description'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [24]:
all_words = df1['university'].str.split(expand=True).unstack().value_counts()
data = [go.Bar(
            x = all_words.index.values[2:50],
            y = all_words.values[2:50],
            marker= dict(colorscale='Jet',
                         color = all_words.values[2:100]
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 (Uncleaned) Word frequencies in the University'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

In [25]:
stopwords_new = ['universidad','programa',
                 'formacion','desarrollo','profesionales',
                 'colombia', 'estudiante', 'quindio', 'cooperativa', 'santo', 'tomas',
                 'sergio','arboleda','pontificia','bolivariana']

In [26]:
nltk.download('punkt')
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')
spanish_stopwords.extend(stopwords_new)

def tokenize(sentence):
    return [token for token in nltk.word_tokenize(sentence)]
def remove_stopwords(sentence):
    return [token for token in nltk.word_tokenize(sentence) if (token.lower() not in spanish_stopwords) and (token.lower() !=' ') and (token not in string.punctuation)]

In [27]:
df1['token'] = df1['description'].apply(lambda x: tokenize(x))
df1['token_no_stopwords'] = df1['description'].apply(lambda x: remove_stopwords(x))
df1['bigram'] = df1['token_no_stopwords'].apply(lambda x: list(ngrams(x, 2)))

In [28]:
df1.head()

In [29]:
vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
X = vectorizer.fit_transform(df1['description'])
model = KMeans( init='k-means++', max_iter=400, random_state=2021,)

In [30]:
X.shape

In [31]:
from yellowbrick.cluster import KElbowVisualizer

# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,10), timings= True,)
visualizer.fit(X)        # Fit data to visualizer
visualizer.show()        # Finalize and render figure
plt.show()

In [32]:
visualizer = KElbowVisualizer(model, k=(2,10), metric='silhouette', timings= True)
visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure
plt.show()

In [33]:
n_clusters = 7

def get_clusters_top_words(n_clusters):
    model = KMeans(n_clusters, init='k-means++', max_iter=400, random_state=2021)
    model.fit(X)
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()

    for i in range(n_clusters):
        print('Cluster %d:' % i),
        for ind in order_centroids[i, :5]:
            print(' %s' % terms[ind])

get_clusters_top_words(7)

In [34]:
get_clusters_top_words(3)

In [35]:
from collections import Counter

bigram_list = [pair for row in df1['token_no_stopwords'] for pair in ngrams(row, 2)]
bigram = Counter(bigram_list).most_common()
bigram = pd.DataFrame.from_records(bigram, columns=['gram', 'count'])
bigram[:20]

In [36]:
words = (df1['token_no_stopwords'].apply(lambda x: ' '.join(x))).str.cat(sep=' ').split()
Counter(words).most_common(50)

In [37]:
import re
data = df1['token_no_stopwords'].str.join(' ').values.tolist()
data = [re.sub('\s+', ' ', sent) for sent in data] # Remove new line characters
data = [re.sub("\'", "", sent) for sent in data] # Remove distracting single quotes

In [38]:
len(data)

In [39]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [40]:
def sentence_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sentence_to_words(data))
print(data_words[:1])

# Creating Bigram and Trigram Models

In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

The bigrams model is ready. Let’s define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [42]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in spanish_stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [43]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
!python -m spacy download es_core_news_sm

In [45]:
import es_core_news_sm

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = es_core_news_sm.load()

# Do lemmatization
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])

# Create the Dictionary and Corpus needed for Topic Modeling

In [46]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [47]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:5]]

# Building the Topic Model

* We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

* Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

* chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, # corpus
                                           id2word=id2word, # index to word
                                           num_topics=7,  # 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# View the topics in LDA model

In [51]:
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning,module='gensim') 

In [53]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
best_model = None
top_score = 0
for x in range(1,16):
    print(f'Number of topics:{x}')
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=x, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    # Compute Perplexity
    print(f'Perplexity for {x} topics: {lda_model.log_perplexity(corpus)}')  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f'Coherence Score for {x} topics: {coherence_lda} \n')
    if coherence_lda > top_score:
        best_model = x
        top_score = coherence_lda
print(f'\nBest Results with {best_model} topics with a Coherence of {top_score}')

**Topic 14**
* Perplexity:  -7.653208814753565

* Coherence Score:  0.40041877333075954

**There we have a coherence score of 0.40.**

# Compute Model Perplexity and Coherence Score

Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [54]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics-keywords using pyLDAvis

In [55]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [76]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

# University Recommender


In [56]:
norm_corpus = data
norm_corpus[:2]

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

## Compute Pairwise Document Similarity

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

In [59]:
df1.head(2)

## Get List of Undergraduates

In [64]:
underg_list = df1['undergraduate'].values
underg_list[:5], underg_list.shape

# Find Top Similar Undergraduate for an Undergraduate Core

Let's take `derecho` the most popular underg the the dataframe above and try and find the most similar undergraduates which can be recommended

In [65]:
underg_idx = np.where(underg_list == 'derecho')[0][0]
underg_idx

**Get similarities**

In [66]:
underg_similarities = doc_sim_df.iloc[underg_idx].values
underg_similarities

**Get top 5 IDs**

In [67]:
similar_underg_idxs = np.argsort(-underg_similarities)[1:6]
similar_underg_idxs

**Get top 5 similar undergraduates**

In [68]:
similar_underg = underg_list[similar_underg_idxs]
similar_underg

## Build a movie recommender function to recommend top 5 similar Undergraduate to an Undergraduate

The undergraduate title, undergraduate title list and document similarity matrix dataframe will be given as inputs to the function

In [73]:
def underg_recommender(underg_title, undergs= underg_list, doc_sims=doc_sim_df):
    # find underg id
    underg_idx = np.where(undergs == underg_title)[0][0]
    # get underg similarities
    underg_similarities = doc_sims.iloc[underg_idx].values
    # get top 5 similar underg IDs
    similar_underg_idxs = np.argsort(-underg_similarities)[1:6]
    # get top 5 undergs
    similar_undergs = undergs[similar_underg_idxs]
    # return the top 5 undergs
    return similar_undergs

In [71]:
df1['undergraduate'].unique()

In [82]:
underg = 'administracion'
print('undergraduate:', underg)
print('Top 5 recommended Undergraduates:', underg_recommender(underg_title=underg, undergs=underg_list, doc_sims=doc_sim_df))
